# Lesson 4

Now that we know what JSON is and we have connected to the skyscanner API let's work with it. In this lesson we will create a function for which if we introduce two dates, an origin and a destination we will get the data from the cheapest flight between these two dates.

We will start by creating a function that takes two dates and return a list of dates between the given ones. Luckily, there is a pandas method for that.

In [37]:
def get_dates(start, end):
    return pd.Series(pd.date_range(start, end,freq='d').format())

get_dates('1992-01-01','1992-01-20')

0     1992-01-01
1     1992-01-02
2     1992-01-03
3     1992-01-04
4     1992-01-05
5     1992-01-06
6     1992-01-07
7     1992-01-08
8     1992-01-09
9     1992-01-10
10    1992-01-11
11    1992-01-12
12    1992-01-13
13    1992-01-14
14    1992-01-15
15    1992-01-16
16    1992-01-17
17    1992-01-18
18    1992-01-19
19    1992-01-20
dtype: object

In [38]:
def city_code(city_name):
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/US/USD/en-US/"

    params = {"query": city_name}

    headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
                      'x-rapidapi-key': x_rapidapi_key}

    response = requests.get(url, headers = headers, params = params)

    return response.json()["Places"][0]["PlaceId"]

In [39]:
city_code('London')

'LOND-sky'

Let's use the funtion to get the flights between two destinations on a given date.

In [40]:
def flight_prices(departure, arrival, date):

    departure_code = city_code(departure)
    arrival_code = city_code(arrival)
    url = f"https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/{departure_code}/{arrival_code}/{date}"

    params = {"inboundpartialdate":{date}}

    #headers = {
    #'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    #'x-rapidapi-key': "<introduce your RapidAPI key here>"}

    headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': x_rapidapi_key}

    response = requests.get(url, headers=headers, params=params)

    return response.json()

flight_prices('Paris', 'London','2020-12-10')

{'Quotes': [{'QuoteId': 1,
   'MinPrice': 61,
   'Direct': True,
   'OutboundLeg': {'CarrierIds': [881],
    'OriginId': 44759,
    'DestinationId': 65698,
    'DepartureDate': '2020-12-10T00:00:00'},
   'QuoteDateTime': '2020-11-24T11:03:00'}],
 'Carriers': [{'CarrierId': 881, 'Name': 'British Airways'}],
 'Places': [{'Name': 'Paris Charles de Gaulle',
   'Type': 'Station',
   'PlaceId': 44759,
   'IataCode': 'CDG',
   'SkyscannerCode': 'CDG',
   'CityName': 'Paris',
   'CityId': 'PARI',
   'CountryName': 'France'},
  {'Name': 'London Heathrow',
   'Type': 'Station',
   'PlaceId': 65698,
   'IataCode': 'LHR',
   'SkyscannerCode': 'LHR',
   'CityName': 'London',
   'CityId': 'LOND',
   'CountryName': 'United Kingdom'},
  {'Name': 'Paris',
   'Type': 'City',
   'PlaceId': 3579769,
   'SkyscannerCode': 'PARI',
   'CityId': 'PARI',
   'CountryName': 'France'}],
 'Currencies': [{'Code': 'USD',
   'Symbol': '$',
   'ThousandsSeparator': ',',
   'DecimalSeparator': '.',
   'SymbolOnLeft': Tr

Now that we already have our date function let's create another function that takes the origin, destination and dates as arguments. It will create a dictionary that has every day between the two dates and containts a call to the API with all the information about the days:

In [41]:
from  time import sleep

def flight_days(origin, destination, start, end):
    dates = get_dates(start, end)
    my_dict = {}
    for date in list(dates):
        #result = None
        #while (result == None):
            try: 
                print(date)
                print("Gathering fligths")
                result = flight_prices(origin, destination, date)
                my_dict[date] = result
            except: 
                print("Error from the API")
                sleep(60)
    return my_dict

flight_days('Paris','Barcelona','2020-11-26','2020-12-10')

2020-11-26
Gathering fligths
2020-11-27
Gathering fligths
2020-11-28
Gathering fligths
2020-11-29
Gathering fligths
2020-11-30
Gathering fligths
2020-12-01
Gathering fligths
2020-12-02
Gathering fligths
2020-12-03
Gathering fligths
2020-12-04
Gathering fligths
2020-12-05
Gathering fligths
2020-12-06
Gathering fligths
2020-12-07
Gathering fligths
2020-12-08
Gathering fligths
2020-12-09
Gathering fligths
2020-12-10
Gathering fligths


{'2020-11-26': {'Quotes': [{'QuoteId': 1,
    'MinPrice': 50,
    'Direct': False,
    'OutboundLeg': {'CarrierIds': [1090],
     'OriginId': 43313,
     'DestinationId': 42414,
     'DepartureDate': '2020-11-26T00:00:00'},
    'QuoteDateTime': '2020-11-21T09:15:00'},
   {'QuoteId': 2,
    'MinPrice': 129,
    'Direct': True,
    'OutboundLeg': {'CarrierIds': [1218],
     'OriginId': 73097,
     'DestinationId': 42414,
     'DepartureDate': '2020-11-26T00:00:00'},
    'QuoteDateTime': '2020-11-24T11:30:00'}],
  'Carriers': [{'CarrierId': 1090, 'Name': 'Ryanair'},
   {'CarrierId': 1218, 'Name': 'Iberia'}],
  'Places': [{'Name': 'Barcelona',
    'Type': 'Station',
    'PlaceId': 42414,
    'IataCode': 'BCN',
    'SkyscannerCode': 'BCN',
    'CityName': 'Barcelona',
    'CityId': 'BARC',
    'CountryName': 'Spain'},
   {'Name': 'Paris Beauvais',
    'Type': 'Station',
    'PlaceId': 43313,
    'IataCode': 'BVA',
    'SkyscannerCode': 'BVA',
    'CityName': 'Paris',
    'CityId': 'PARI',
 

Now let's create another function that returns the date of the cheapest flight.

In [42]:
def lowest_price(origin, destination, start, end):
    flights = flight_days(origin, destination, start, end)
    prices = {date:min([quote["MinPrice"] for quote in flights[date]["Quotes"]]) for date in flights for quote in flights[date]["Quotes"]}
    return min(prices, key = prices.get)

The problem with this code is that it just gets the first date as the only one even if there are other days with the same price. Try to fix that in the lab.

In [44]:
prices = {date:min([quote["MinPrice"] for quote in flights[date]["Quotes"]]) for date in flights for quote in flights[date]["Quotes"]}
min(prices, key = prices.get)

'2020-11-29'

# Lab 2


Following the class example, create a function that returns the price, names of origin and arrival airports and the name of the company. Do it for all the flights between two dates that cost the same.

In [45]:
prices

{'2020-11-26': 50,
 '2020-11-27': 88,
 '2020-11-28': 48,
 '2020-11-29': 18,
 '2020-11-30': 31,
 '2020-12-01': 87,
 '2020-12-02': 18,
 '2020-12-03': 61,
 '2020-12-04': 75,
 '2020-12-05': 55,
 '2020-12-06': 18,
 '2020-12-07': 62,
 '2020-12-08': 87,
 '2020-12-09': 18,
 '2020-12-10': 33}

In [49]:
# revese key/value
rev_multidict={} 
for key, value in prices.items():
    rev_multidict.setdefault(value, set()).add(key)
rev_multidict

{50: {'2020-11-26'},
 88: {'2020-11-27'},
 48: {'2020-11-28'},
 18: {'2020-11-29', '2020-12-02', '2020-12-06', '2020-12-09'},
 31: {'2020-11-30'},
 87: {'2020-12-01', '2020-12-08'},
 61: {'2020-12-03'},
 75: {'2020-12-04'},
 55: {'2020-12-05'},
 62: {'2020-12-07'},
 33: {'2020-12-10'}}

In [56]:
days_min =[values for key, values in rev_multidict.items() if key == min(rev_multidict.keys())]


[{'2020-11-29', '2020-12-02', '2020-12-06', '2020-12-09'}]

In [63]:
def lowest_price_days(origin, destination, start, end):
    flights = flight_days(origin, destination, start, end)
    prices = {date:min([quote["MinPrice"] for quote in flights[date]["Quotes"]]) for date in flights for quote in flights[date]["Quotes"]}
    rev_multidict={} 
    for key, value in prices.items():
        rev_multidict.setdefault(value, set()).add(key)
    print("Min Price Days are :")
    return [values for key, values in rev_multidict.items() if key == min(rev_multidict.keys())]

In [64]:
lowest_price_days('Paris','Barcelona','2020-11-26','2020-12-10')

2020-11-26
Gathering fligths
2020-11-27
Gathering fligths
2020-11-28
Gathering fligths
2020-11-29
Gathering fligths
2020-11-30
Gathering fligths
2020-12-01
Gathering fligths
2020-12-02
Gathering fligths
2020-12-03
Gathering fligths
2020-12-04
Gathering fligths
2020-12-05
Gathering fligths
2020-12-06
Gathering fligths
2020-12-07
Gathering fligths
2020-12-08
Gathering fligths
2020-12-09
Gathering fligths
2020-12-10
Gathering fligths
Min Price Days are :


[{'2020-11-29', '2020-12-02', '2020-12-06', '2020-12-09'}]